In [1]:
!pip install neurokit2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.5/696.5 kB 9.4 MB/s eta 0:00:00:00:0100:01


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import json
import itertools
from tqdm import tqdm
import neurokit2 as nk

In [3]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [4]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [5]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

In [6]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

In [7]:
lambda_max = {
    "lorenz": 0.9056  # Example ground truth LE for the Lorenz system
}

In [17]:
import neurokit2 as nk
lambda_max = {
        "Mackey": 0.006100,
        "Lorenz": 0.905600,
        "Rossler": 0.071400,
        "Chen": 0.829600,
        "Chua": 0.428400
        }
def compute_lyapunov_exponent(chosen_system, trajectory, dt):
    """
    Compute the Lyapunov Exponent for a given trajectory using NeuroKit2.

    Parameters:
        chosen_system (str): Name of the dynamical system (for comparison with true LEs).
        trajectory (np.ndarray): Shape (N,) or (N, 3) trajectory.
        dt (float): Time step used in the simulation.

    Returns:
        lyapunov_exponent (float): Estimated max Lyapunov exponent.
        diff (float): Absolute error from ground truth value.
    """
    trajectory = np.asarray(trajectory)

    # Ensure it's 2D for consistency
    if trajectory.ndim == 1:
        trajectory = trajectory.reshape(-1, 1)

    # # Compute delay and dimension from the first component
    # delay, _ = nk.complexity_delay(signal=trajectory[:, 0])
    # dimension, _ = nk.complexity_dimension(signal=trajectory[:, 0], delay=delay)

    # Compute LE for each dimension
    les = []
    for i in range(trajectory.shape[1]):
        le, _ = nk.complexity_lyapunov(signal=trajectory[:, i])
        les.append(le)

    lyapunov_exponent = max(les)
    diff = np.abs(lyapunov_exponent - lambda_max[chosen_system])

    return diff

In [9]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

class HFRRes3D:
    """
    Hierarchical Fractal Reservoir (HFR) for 3D chaotic systems.
    
    This novel reservoir architecture partitions the chaotic attractor at multiple
    hierarchical scales, combining them in a fractal-like adjacency structure.
    The method is model-free, relying solely on the observed trajectory in R^3,
    and does not require knowledge of any system parameters such as sigma, rho, beta
    for Lorenz63. 
    
    Key Idea:
     1) Define multiple 'scales' of partition of the data's bounding region.
     2) Each scale is subdivided into a certain number of cells (regions).
     3) Each cell at level l has links to both:
        - other cells at the same level (horizontal adjacency),
        - 'child' cells at the finer level l+1 (vertical adjacency).
     4) We gather all cells across levels => a multi-level fractal graph => adjacency => W.
     5) We build a typical ESN from this adjacency, feed data with W_in, run leaky tanh updates,
        then do a polynomial readout for 3D next-step prediction.

    This approach is suitable for chaotic systems whose attractors often exhibit fractal
    self-similarity, thus capturing multi-scale structures in a single reservoir.
    """

    def __init__(self,
                 n_levels=3,             # number of hierarchical levels
                 cells_per_level=None,   # list of number of cells at each level, e.g. [8, 32, 128]
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        """
        Parameters
        ----------
        n_levels       : int, number of hierarchical scales
        cells_per_level: list[int], the number of partitions/cells at each level
                         if None, we auto-generate e.g. 2^(level+2)
        spectral_radius: final scaling for adjacency
        input_scale    : random input scale W_in
        leaking_rate   : ESN leaky alpha
        ridge_alpha    : readout ridge penalty
        seed           : random seed
        """
        self.n_levels        = n_levels
        self.cells_per_level = cells_per_level
        self.spectral_radius = spectral_radius
        self.input_scale     = input_scale
        self.leaking_rate    = leaking_rate
        self.ridge_alpha     = ridge_alpha
        self.seed            = seed

        if self.cells_per_level is None:
            # default scheme e.g. 8, 16, 32 for 3 levels
            self.cells_per_level = [8*(2**i) for i in range(n_levels)]

        # We'll store adjacency W, input W_in, readout W_out, reservoir state x
        self.W     = None
        self.W_in  = None
        self.W_out = None
        self.x     = None
        self.n_levels = len(self.cells_per_level)

        # We'll define a total number of nodes = sum(cells_per_level)
        self.n_nodes = sum(self.cells_per_level)

    def _build_partitions(self, data_3d):
        """
        Build hierarchical partitions for each level.
        We'll store the bounding box for data_3d, then for each level l in [0..n_levels-1]
        run e.g. k-means with K = cells_per_level[l], each point gets a label => we track transitions.

        Return: 
          partitions => list of arrays, partitions[l] => shape (N, ) cluster assignment in [0..cells_per_level[l]-1]
        """
        from sklearn.cluster import KMeans
        N = len(data_3d)
        partitions = []

        for level in range(self.n_levels):
            k = self.cells_per_level[level]
            # cluster
            kmeans = KMeans(n_clusters=k, random_state=self.seed+10*level, n_init='auto')
            kmeans.fit(data_3d)
            labels = kmeans.predict(data_3d)
            partitions.append(labels)

        return partitions

    def _build_hierarchical_adjacency(self, data_3d):
        """
        Build a block adjacency with cross-level links, then scale spectral radius.
        Steps:
          1) Build partitions for each level => partitions[l] in [0..cells_per_level[l]-1]
          2) For each level l, build a transition matrix T_l of shape (cells_per_level[l], cells_per_level[l]).
          3) Link scale l to scale l+1 by figuring out which cluster i at scale l maps to which cluster j at scale l+1
             for each sample t => link i-> j if data_3d[t] is in i at scale l and j at scale l+1.
          4) Combine all transitions in one big adjacency W in R^(n_nodes x n_nodes).
          5) row-normalize W => scale largest eigenvalue => spectral_radius
        """
        partitions = self._build_partitions(data_3d)
        N = len(data_3d)

        # offsets for each level => to index big W
        offsets = []
        running = 0
        for level in range(self.n_levels):
            offsets.append(running)
            running += self.cells_per_level[level]

        # total nodes
        n_tot = self.n_nodes
        # initialize adjacency
        A = np.zeros((n_tot, n_tot))

        # 1) horizontal adjacency in each level
        for level in range(self.n_levels):
            k = self.cells_per_level[level]
            labels = partitions[level]
            # T_l => shape (k, k)
            T_l = np.zeros((k, k))
            for t in range(N-1):
                i = labels[t]
                j = labels[t+1]
                T_l[i,j]+=1
            # row normalize
            row_sum = T_l.sum(axis=1, keepdims=True)
            row_sum[row_sum==0.0] = 1.0
            T_l /= row_sum
            # place T_l into big A
            off = offsets[level]
            A[off:off+k, off:off+k] = T_l

        # 2) vertical adjacency between scale l and l+1
        for level in range(self.n_levels-1):
            k_l   = self.cells_per_level[level]
            k_lp1 = self.cells_per_level[level+1]
            labels_l   = partitions[level]
            labels_lp1 = partitions[level+1]
            # we define adjacency from i in [0..k_l-1] to j in [0..k_lp1-1] if the same sample t belongs to i at level l and j at l+1
            # Count how many times
            Xvert1 = np.zeros((k_l, k_lp1))
            for t in range(N):
                i = labels_l[t]
                j = labels_lp1[t]
                Xvert1[i,j]+=1
            # row normalize
            row_sum = Xvert1.sum(axis=1, keepdims=True)
            row_sum[row_sum==0.0] = 1.0
            Xvert = Xvert1/row_sum
            # place in big A
            off_l   = offsets[level]
            off_lp1 = offsets[level+1]
            A[off_l:off_l+k_l, off_lp1:off_lp1+k_lp1] = Xvert
            # tentative idea, we could also define adjacency from l+1 -> l (parent link), if desired
            # we do the same for the 'child -> parent' link or skip it if we only want forward adjacency
            # For now, let's do symmetrical
            Yvert = Xvert1.T
            col_sum = Yvert.sum(axis=1, keepdims=True)
            col_sum[col_sum==0.0] = 1.0
            Yvert /= col_sum
            A[off_lp1:off_lp1+k_lp1, off_l:off_l+k_l] = Yvert

        # now we have a big adjacency => row normalize again, then scale spectral radius
        row_sum = A.sum(axis=1, keepdims=True)
        row_sum[row_sum==0.0] = 1.0
        A /= row_sum

        A = scale_spectral_radius(A, self.spectral_radius)
        return A

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Main training routine:
          1) Build hierarchical adjacency from fractal partition => self.W
          2) define W_in => shape(n_nodes, 3)
          3) teacher forcing => polynomial readout => solve => self.W_out
        """
        np.random.seed(self.seed)
        # Build adjacency
        W_big = self._build_hierarchical_adjacency(train_input)
        self.W = W_big

        # define W_in => shape(n_nodes,3)
        self.n_nodes = W_big.shape[0]
        self.W_in = (np.random.rand(self.n_nodes,3)-0.5)*2.0*self.input_scale

        # define reservoir state
        self.x = np.zeros(self.n_nodes)

        # gather states => teacher forcing => polynomial => readout
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]
        X_list= []
        for s in states_use:
            X_list.append( augment_state_with_squares(s) )
        X_aug= np.array(X_list)

        reg= Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        self.W_out= reg.coef_

    def collect_states(self, inputs, discard=100):
        """
        Teacher forcing => feed real 3D => gather states => shape => [T-discard, n_nodes].
        returns (states_after_discard, states_discarded).
        """
        self.reset_state()
        states= []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states= np.array(states)
        return states[discard:], states[:discard]

    def reset_state(self):
        if self.x is not None:
            self.x.fill(0.0)

    def _update(self, u):
        """
        x(t+1)= (1-alpha)x(t)+ alpha tanh( W*x(t)+ W_in*u(t) ).
        """
        alpha= self.leaking_rate
        pre_acts= self.W@self.x + self.W_in@u
        x_new= np.tanh(pre_acts)
        self.x= (1.0- alpha)*self.x+ alpha*x_new

    def predict_autoregressive(self, initial_input, n_steps):
        """
        fully autonomous => feed last predicted => next input
        """
        preds= []
        #self.reset_state()
        current_in= np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            big_x= augment_state_with_squares(self.x)
            out= self.W_out@big_x
            preds.append(out)
            current_in= out
        return np.array(preds)
    
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

In [13]:
grid = {
    "cells_per_level": [[5, 15, 20, 30, 45, 50, 55, 80]],
    "spectral_radius": [0.92],
    "input_scale": [1],
    "leaking_rate": [0.9],
    "ridge_alpha": [1e-8]
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # horizons = [200, 400, 600, 800, 1000]

    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores_vpt = []
        # horizon_nrmse_all = {h: [] for h in horizons}
        adev_scores = []
        # ldev_scores = []

        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt = 0.02
            t_vals, lorenz_traj = generate_lorenz_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )

            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]

            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)

            T_data = len(lorenz_traj)
            for train_frac in [0.7, 0.75, 0.8]:
                train_end = int(train_frac * (T_data - 1))
                train_input = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end + 1]
                test_input = lorenz_traj[train_end:-1]
                test_target = lorenz_traj[train_end + 1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]

                for seed in np.arange(1, 6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)

                    T_VPT_s, _, _ = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, 0.9, dt)
                    seed_scores_vpt.append(T_VPT_s)

                    # horizon_nrmse = evaluate_nrmse(preds, test_target, horizons)
                    # for h in horizons:
                    #     horizon_nrmse_all[h].append(horizon_nrmse[h])

                    adev = compute_attractor_deviation(preds, test_target)
                    adev_scores.append(adev)

                    # ldev = compute_lyapunov_exponent("Lorenz", preds, dt)
                    # ldev_scores.append(ldev)

        mean_vpt = float(np.mean(seed_scores_vpt))
        std_vpt = float(np.std(seed_scores_vpt))
        # mean_nrmse_dict = {str(h): float(np.mean(horizon_nrmse_all[h])) for h in horizons}
        # std_nrmse_dict  = {str(h): float(np.std(horizon_nrmse_all[h]))  for h in horizons}
        mean_adev = float(np.mean(adev_scores))
        std_adev = float(np.std(adev_scores))
        # mean_ldev = float(np.mean(ldev_scores))
        # std_ldev = float(np.std(ldev_scores))

        results.append({
            "params": params,
            "seed_scores_T_VPT": seed_scores_vpt,
            "mean_T_VPT": mean_vpt,
            "std_T_VPT": std_vpt,
            # "mean_NRMSEs": mean_nrmse_dict,
            # "std_NRMSEs": std_nrmse_dict,
            "mean_ADev": mean_adev,
            "std_ADev": std_adev,
            # "mean_LDev": mean_ldev,
            # "std_LDev": std_ldev
        })

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")

    return results

In [18]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    # Precompute param combinations
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # tqdm adds a progress bar for better visualization
    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores = []
        adevs=[]
        lyups=[]
        
        # Run all 20 seeds
        for initial_state in [[1.0,1.0,1.0],[1.0,2.0,3.0],[2.0,1.5,4.0]]:
            tmax = 250
            dt   = 0.02
            t_vals, lorenz_traj = generate_lorenz_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )
            
            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]
            
            # normalize
            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)
            
            T_data = len(lorenz_traj)
            for train_frac in [0.7,0.75,0.8]:
                train_end = int(train_frac*(T_data-1))
                train_input  = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end+1]
                test_input   = lorenz_traj[train_end:-1]
                test_target  = lorenz_traj[train_end+1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]
                for seed in np.arange(1,6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    T_VPT_s, _, _ = compute_valid_prediction_time(test_target,preds,t_vals,0.4,0.9,dt)
                    seed_scores.append(T_VPT_s)
                    adev= compute_attractor_deviation(preds, test_target, (0.1,0.1,0.1))
                    adevs.append(adev)
                    lyup = compute_lyapunov_exponent("Lorenz", preds, dt)
                    lyups.append(lyup)
        mean_score = float(np.mean(seed_scores))
        std_dev    = float(np.std(seed_scores))
        is_stable  = std_dev < 1.5
        status     = "Stable" if is_stable else "Unstable"
        mean_adev= float(np.mean(adevs))
        mean_lyup=float(np.mean(lyups))
        std_dev_adev = float(np.std(adevs))
        std_dev_lyup = float(np.std(lyups))
        
        # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
        #       f"Std Dev={std_dev:.3f} → {status}")

        print(f"Mean VPT: {mean_score} Std Dev VPT: {std_dev}" 
        f"Adev: {mean_adev} Std Dev Adev: {std_dev_adev}" 
        f"Lyupanov: {mean_lyup} Std Dev Lyupanov: {std_dev_lyup}")

        results.append({
            "params":      params,
            "seed_scores": seed_scores,
            "mean_T_VPT":  mean_score,
            "std_dev":     std_dev,
            "LDev": mean_lyup + std_dev_lyup
        })

    # Save results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to {output_path}")
    
    return results

In [19]:
run_grid_search(HFRRes3D, grid, "HFR", output_path="hfr_grid_search_results.json")


== Initial grid search for HFR with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:51<00:00, 51.39s/it]

Mean VPT: 12.831555555555555 Std Dev VPT: 1.6508825324057634Adev: 26.844444444444445 Std Dev Adev: 11.191178183542718Lyupanov: 0.8010989481322953 Std Dev Lyupanov: 0.005840453690493894

All results saved to hfr_grid_search_results.json


[{'params': {'cells_per_level': [5, 15, 20, 30, 45, 50, 55, 80],
   'spectral_radius': 0.92,
   'input_scale': 1,
   'leaking_rate': 0.9,
   'ridge_alpha': 1e-08},
  'seed_scores': [12.759999999999998,
   14.880000000000003,
   12.660000000000004,
   12.64,
   15.660000000000004,
   11.259999999999998,
   12.800000000000004,
   11.14,
   12.800000000000004,
   14.32,
   11.420000000000002,
   13.0,
   13.740000000000002,
   12.899999999999999,
   12.800000000000004,
   11.480000000000004,
   13.780000000000001,
   11.520000000000003,
   16.08,
   11.480000000000004,
   12.280000000000001,
   12.340000000000003,
   13.82,
   14.0,
   12.240000000000002,
   12.160000000000004,
   12.259999999999998,
   12.18,
   16.92,
   12.18,
   13.259999999999998,
   13.020000000000003,
   13.04,
   13.200000000000003,
   18.08,
   12.219999999999999,
   12.120000000000005,
   12.14,
   12.020000000000003,
   15.600000000000001,
   10.560000000000002,
   10.660000000000004,
   10.64,
   10.7400000000